In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from scipy import stats
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
#----------------------
from dataset import CreateDataBatches
from models import Model
from utils_cm import compute_cm, split_dataset

## Generating Synthetic Data

In [ ]:
### Generate synthetic data using Gaussians
sizes = [1000, 1000]

# class0:
mu0 = np.array([-1, 7])
cov0 = np.array([[1, 0], [0, 1]]) * 0.15
data0 = np.random.multivariate_normal(mu0, cov0, size=sizes[0])

# class1:
mu1 = np.array([5, -3])
cov1 = np.array([[10, 0], [0, 10]]) * 0.5
data1 = np.random.multivariate_normal(mu1, cov1, size=sizes[1])

### Combine data from different classes, shuffle them and split it into train, validation and test sets
data = np.vstack([data0, data1])
labels = np.concatenate([i * np.ones(sizes[i]) for i in range(len(sizes))]).astype(int)
N = sum(sizes)
split = [0.6, 0.8, 1]
data_train, labels_train, data_val, labels_val, data_test, labels_test = split_dataset(data, labels, split)
data_mean = data_train.mean(axis=0)
data_std = data_train.std(axis=0)

classes = np.unique(labels)
class_colours = ['r', 'b']

idx_train = [np.where(labels_train == c)[0] for c in classes]
idx_test = [np.where(labels_test == c)[0] for c in classes]

### Plot the Training Data

In [ ]:
for c in classes:
    plt.scatter(data_train[idx_train[c], 0], data_train[idx_train[c], 1], marker='.', s=100, color=class_colours[c])

## Prepare the Dataset

In [ ]:
normalize = True
trainloader = DataLoader(CreateDataBatches(data_train, labels_train, data_mean, data_std, normalize=normalize), batch_size=16, shuffle=True)
valloader = DataLoader(CreateDataBatches(data_val, labels_val, data_mean, data_std, normalize=normalize), batch_size=16, shuffle=False)
testloader = DataLoader(CreateDataBatches(data_test, labels_test, data_mean, data_std, normalize=normalize), batch_size=16, shuffle=False)

## Design the Model

In [ ]:
model = Model(input_size=data_train.shape[-1], nclasses=len(classes), hidden_layers=[8], dropout=0.5)
model.net

## Configure the Training step

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
model.to(device)
print(device)

## Training

In [ ]:
loss_value = 0
n_epochs = 1
reset_loss_every = 10

model.train()
for epoch in range(n_epochs):
    for it, train_batch in enumerate(trainloader):
        model.train()
        train_data_batch, train_labels_batch = train_batch
        output = model(train_data_batch.to(device).float())
        optim.zero_grad()
        loss = F.cross_entropy(output, train_labels_batch.to(device), reduction="mean")
        loss.backward()
        loss_value += loss.data.item()
        optim.step()

        if it % reset_loss_every == 0 and it > 0:
            model.eval()
            gt_val, preds_val = [], []
            for it_val, val_batch in enumerate(valloader):
                val_data_batch, val_labels_batch = val_batch
                output_val = model(val_data_batch.to(device).float())
                preds_val.append(F.softmax(output_val, dim=1).data.numpy().argmax(axis=1))
                gt_val.append(val_labels_batch.numpy())
            preds_val = np.hstack(preds_val)
            gt_val = np.hstack(gt_val)
            recall, precision = compute_cm(gt_val, preds_val, classes)
            average_loss = np.round(loss_value / reset_loss_every, 4)
            print(f'epoch: {epoch}, iteration: {it}, recall: {recall},  precision: {precision}, average_loss: {average_loss}')
            loss_value = 0


## Classify all grid points to visualize decision boundaries

In [ ]:
axis0_min = data_test[:, 0].min()
axis0_max = data_test[:, 0].max()
axis1_min = data_test[:, 1].min()
axis1_max = data_test[:, 1].max()
range_0 = np.arange(axis0_min, axis0_max, .1)
range_1 = np.arange(axis1_min, axis1_max, .1)
data_grid = np.array([(x0, x1) for x0 in range_0
                               for x1 in range_1])
gridloader = DataLoader(CreateDataBatches
                        (data_grid, 0*data_grid[:, 0], data_mean, data_std, normalize=normalize), 
                         batch_size=16, shuffle=False)
# stats.describe(data_grid)

In [ ]:
model.eval()
preds_grid = []
for it_grid, grid_batch in enumerate(gridloader):
    grid_data_batch, _ = grid_batch
    output_grid = model(grid_data_batch.to(device).float())
    preds_grid.append(F.softmax(output_grid, dim=1).data.numpy())
preds_grid = np.vstack(preds_grid)
preds_grid_score = preds_grid.max(-1)
# preds_grid_score = preds_grid[:, 1]
preds_grid_labels = preds_grid.argmax(-1)

## Evaluating on the test set

In [ ]:
model.eval()
gt_test, preds_test = [], []
for it_test, test_batch in enumerate(testloader):
    test_data_batch, test_labels_batch = test_batch
    output_test = model(test_data_batch.to(device).float())
    preds_test.append(F.softmax(output_test, dim=1).data.numpy())
    gt_test.append(test_labels_batch.numpy())
preds_test = np.vstack(preds_test)
preds_test_labels = preds_test.argmax(-1)
preds_test_score = preds_test.max(-1)
gt_test = np.hstack(gt_test)
recall, precision = compute_cm(gt_test, preds_test_labels, classes)
print(f'recall: {recall},  precision: {precision}')
preds_test.shape

In [ ]:
fig=plt.figure(figsize=(18, 16))

score_grid = preds_grid_score.reshape((len(range_0), len(range_1)))
plt.imshow(score_grid.T, origin='lower', aspect='auto', extent=(axis0_min, axis0_max, axis1_min, axis1_max))
plt.colorbar()

# for c in classes:
#     plt.scatter(data_grid[preds_grid_labels==c, 0], data_grid[preds_grid_labels==c, 1], 
#                 marker='.', s=100, color=class_colours[c], alpha = 0.05)

markers = ['s', 'o']
for c in classes:
    plt.scatter(data_test[gt_test==c, 0], data_test[gt_test==c, 1], 
                marker=markers[c], edgecolors='k', s=100, color=class_colours[c])